In [2]:
from gensim.models.doc2vec import Doc2Vec

In [3]:
def WordCut(seqs, k=1):
    # 分词，分为[['C', 'C', 'C', 'U'],[U', 'G', 'U'],['G', 'C', 'C', 'U', 'U', 'C']]
    res = []
    maxLength = 0
    for seq in seqs:
        cutSequence = []
        for i in range(len(seq) - k + 1):
            cutSequence.append(seq[i:i + k])
        res.append(cutSequence)
        if len(seq) > maxLength:
            maxLength = len(seq)
    return res, maxLength


def Doc2Vector(sequences, k_mer=3, vector_size=50, epochs=30):
    cutSequences, sequenceMaxLength = WordCut(sequences, k_mer)
    model = Doc2Vec(cutSequences, vector_size=vector_size, epochs=epochs)

    encodingsVec = []
    sequencesLength = len(sequences)
    one_percent = max(1, sequencesLength // 100)
    progressIndex = 0
    for cutSentence in cutSequences:
        seqVec = [model.wv[word] for word in cutSentence]
        len_seq_vec = len(seqVec)
        seqVec.extend([np.zeros(vector_size)] * (sequenceMaxLength - len_seq_vec))
        encodingsVec.append(seqVec)
        progressIndex += 1
        if progressIndex % one_percent == 0 or progressIndex == sequencesLength:
            my_emit(signal.progressBar, 100 * progressIndex / sequencesLength)
    encodings = np.array(encodingsVec, dtype=np.float32)
    return encodings.reshape(len(encodings), -1)
